# 🎯 Reinforcement Learning for Prompt Optimization in Agentic Workflows

This notebook demonstrates how to use **Reinforcement Learning (RL)** to dynamically optimize prompts based on user feedback, task performance, and cost efficiency.

## Key Concepts

```
┌──────────────────────────────────────────────────────────────────────────────┐
│              RL-BASED PROMPT OPTIMIZATION SYSTEM                             │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  ┌─────────┐    ┌──────────────────┐    ┌──────────────────┐                │
│  │  User   │───►│  PROMPT AGENT    │───►│   LLM SERVICE    │                │
│  │  Query  │    │                  │    │                  │                │
│  └─────────┘    │ • Select Template│    │ • Generate       │                │
│       │         │ • Add Examples   │    │   Response       │                │
│       │         │ • Set Parameters │    │                  │                │
│       │         └────────┬─────────┘    └────────┬─────────┘                │
│       │                  │                       │                          │
│       │                  ▼                       ▼                          │
│       │         ┌──────────────────┐    ┌──────────────────┐                │
│       │         │  RL OPTIMIZER    │◄───│    RESPONSE      │                │
│       │         │                  │    │                  │                │
│       │         │ • Q-Learning     │    │ • Quality Score  │                │
│       │         │ • Multi-Armed    │    │ • User Feedback  │                │
│       │         │   Bandit         │    │ • Task Success   │                │
│       └────────►│ • Policy Grad    │    │ • Cost Metric    │                │
│   Feedback      └────────┬─────────┘    └──────────────────┘                │
│                          │                                                  │
│                          ▼                                                  │
│                 ┌────────────────────────────────────────────┐              │
│                 │         OPTIMIZED PROMPTS                   │              │
│                 │  Personalized, Efficient, High-Quality      │              │
│                 └────────────────────────────────────────────┘              │
└──────────────────────────────────────────────────────────────────────────────┘
```

## What You'll Learn

1. **Multi-Armed Bandit** for prompt template selection
2. **Q-Learning** for dynamic prompt assembly
3. **Contextual Bandits** for personalized prompts
4. **Policy Gradients** for continuous prompt optimization
5. **RLHF-style** reward modeling from user feedback

## 1. Environment Setup

In [ ]:
# Install required packages
%pip install -q numpy pandas matplotlib seaborn scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Any, Callable
from dataclasses import dataclass, field
from enum import Enum
from collections import defaultdict
import random
from abc import ABC, abstractmethod
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

print("✅ All imports successful!")

## 2. Define Prompt Components and Templates

We define the building blocks of prompts that can be optimized through RL.

In [ ]:
class QueryType(Enum):
    """Types of user queries that require different prompt strategies."""
    FACTUAL = "factual"           # Questions with definitive answers
    CREATIVE = "creative"         # Open-ended creative tasks
    ANALYTICAL = "analytical"     # Data analysis and reasoning
    CODING = "coding"             # Code generation tasks
    CONVERSATIONAL = "conversational"  # Chat-style interactions


class ToneStyle(Enum):
    """Tone options for prompt responses."""
    FORMAL = "formal"
    CASUAL = "casual"
    TECHNICAL = "technical"
    FRIENDLY = "friendly"


class DetailLevel(Enum):
    """Level of detail in responses."""
    BRIEF = "brief"
    MODERATE = "moderate"
    DETAILED = "detailed"
    COMPREHENSIVE = "comprehensive"


@dataclass
class PromptTemplate:
    """A prompt template with configurable components."""
    id: str
    name: str
    base_instruction: str
    supports_examples: bool = True
    supports_chain_of_thought: bool = True
    estimated_cost: float = 1.0  # Relative cost multiplier
    
    def render(self, query: str, examples: List[str] = None, 
               tone: ToneStyle = ToneStyle.FORMAL,
               detail: DetailLevel = DetailLevel.MODERATE,
               chain_of_thought: bool = False) -> str:
        """Render the complete prompt."""
        prompt_parts = [self.base_instruction]
        
        # Add tone instruction
        tone_instructions = {
            ToneStyle.FORMAL: "Use a professional and formal tone.",
            ToneStyle.CASUAL: "Use a casual and conversational tone.",
            ToneStyle.TECHNICAL: "Use precise technical language.",
            ToneStyle.FRIENDLY: "Be warm and approachable in your response."
        }
        prompt_parts.append(tone_instructions[tone])
        
        # Add detail level
        detail_instructions = {
            DetailLevel.BRIEF: "Keep your response concise and to the point.",
            DetailLevel.MODERATE: "Provide a balanced response with key details.",
            DetailLevel.DETAILED: "Provide a thorough and detailed response.",
            DetailLevel.COMPREHENSIVE: "Provide an exhaustive response covering all aspects."
        }
        prompt_parts.append(detail_instructions[detail])
        
        # Add examples if supported and provided
        if self.supports_examples and examples:
            prompt_parts.append("\nExamples:")
            for i, ex in enumerate(examples, 1):
                prompt_parts.append(f"{i}. {ex}")
        
        # Add chain of thought if supported
        if self.supports_chain_of_thought and chain_of_thought:
            prompt_parts.append("\nThink through this step by step before providing your answer.")
        
        prompt_parts.append(f"\nUser Query: {query}")
        
        return "\n".join(prompt_parts)


# Define available prompt templates
PROMPT_TEMPLATES = {
    "standard": PromptTemplate(
        id="standard",
        name="Standard Assistant",
        base_instruction="You are a helpful AI assistant. Answer the user's question accurately and helpfully.",
        estimated_cost=1.0
    ),
    "expert": PromptTemplate(
        id="expert",
        name="Domain Expert",
        base_instruction="You are an expert in the relevant domain. Provide authoritative and well-researched answers.",
        estimated_cost=1.2
    ),
    "teacher": PromptTemplate(
        id="teacher",
        name="Patient Teacher",
        base_instruction="You are a patient teacher. Explain concepts clearly, use analogies, and ensure understanding.",
        estimated_cost=1.3
    ),
    "concise": PromptTemplate(
        id="concise",
        name="Concise Responder",
        base_instruction="You are a concise assistant. Provide direct, no-fluff answers.",
        supports_chain_of_thought=False,
        estimated_cost=0.8
    ),
    "creative": PromptTemplate(
        id="creative",
        name="Creative Writer",
        base_instruction="You are a creative assistant. Think outside the box and provide imaginative solutions.",
        estimated_cost=1.1
    ),
    "analyst": PromptTemplate(
        id="analyst",
        name="Data Analyst",
        base_instruction="You are a data analyst. Approach problems systematically with data-driven insights.",
        estimated_cost=1.4
    )
}

print(f"✅ Defined {len(PROMPT_TEMPLATES)} prompt templates:")
for template_id, template in PROMPT_TEMPLATES.items():
    print(f"   📝 {template.name} (cost: {template.estimated_cost}x)")

## 3. Simulated User Environment and Reward Model

We create a simulated environment that models user preferences and provides feedback signals for RL training.

In [ ]:
@dataclass
class UserProfile:
    """Simulates a user with specific preferences."""
    user_id: str
    preferred_tone: ToneStyle
    preferred_detail: DetailLevel
    preferred_templates: List[str]  # Template IDs user responds well to
    patience: float = 0.5  # 0-1, tolerance for verbose responses
    expertise_level: float = 0.5  # 0-1, technical sophistication
    
    def get_satisfaction(self, template_id: str, tone: ToneStyle, 
                         detail: DetailLevel, response_quality: float) -> float:
        """Calculate user satisfaction based on preferences."""
        satisfaction = response_quality  # Base satisfaction from quality
        
        # Template preference bonus
        if template_id in self.preferred_templates:
            satisfaction += 0.2
        
        # Tone match bonus
        if tone == self.preferred_tone:
            satisfaction += 0.15
        elif tone in [ToneStyle.TECHNICAL, ToneStyle.FORMAL] and self.expertise_level > 0.7:
            satisfaction += 0.1
        elif tone in [ToneStyle.CASUAL, ToneStyle.FRIENDLY] and self.expertise_level < 0.3:
            satisfaction += 0.1
        
        # Detail level preference
        detail_scores = {
            DetailLevel.BRIEF: 0.25,
            DetailLevel.MODERATE: 0.5,
            DetailLevel.DETAILED: 0.75,
            DetailLevel.COMPREHENSIVE: 1.0
        }
        detail_value = detail_scores[detail]
        
        # Users with low patience prefer brief responses
        if self.patience < 0.3 and detail_value > 0.5:
            satisfaction -= 0.2
        # Users with high patience and expertise prefer detailed responses
        elif self.patience > 0.7 and self.expertise_level > 0.6 and detail_value < 0.5:
            satisfaction -= 0.1
        
        if detail == self.preferred_detail:
            satisfaction += 0.1
        
        return np.clip(satisfaction, 0, 1)


@dataclass
class Query:
    """Represents a user query with context."""
    text: str
    query_type: QueryType
    complexity: float  # 0-1
    requires_examples: bool = False
    requires_reasoning: bool = False


class SimulatedLLMEnvironment:
    """Simulates LLM responses and user feedback for RL training."""
    
    def __init__(self):
        self.user_profiles = self._create_user_profiles()
        self.query_bank = self._create_query_bank()
        self.interaction_history = []
        
        # Ground truth: optimal templates per query type (for simulation)
        self.optimal_mappings = {
            QueryType.FACTUAL: ["expert", "concise", "standard"],
            QueryType.CREATIVE: ["creative", "teacher"],
            QueryType.ANALYTICAL: ["analyst", "expert"],
            QueryType.CODING: ["expert", "analyst", "concise"],
            QueryType.CONVERSATIONAL: ["standard", "teacher", "creative"]
        }
    
    def _create_user_profiles(self) -> Dict[str, UserProfile]:
        """Create diverse user profiles."""
        return {
            "tech_expert": UserProfile(
                user_id="tech_expert",
                preferred_tone=ToneStyle.TECHNICAL,
                preferred_detail=DetailLevel.DETAILED,
                preferred_templates=["expert", "analyst"],
                patience=0.8,
                expertise_level=0.9
            ),
            "casual_user": UserProfile(
                user_id="casual_user",
                preferred_tone=ToneStyle.FRIENDLY,
                preferred_detail=DetailLevel.MODERATE,
                preferred_templates=["standard", "teacher"],
                patience=0.4,
                expertise_level=0.3
            ),
            "busy_professional": UserProfile(
                user_id="busy_professional",
                preferred_tone=ToneStyle.FORMAL,
                preferred_detail=DetailLevel.BRIEF,
                preferred_templates=["concise", "expert"],
                patience=0.2,
                expertise_level=0.7
            ),
            "student": UserProfile(
                user_id="student",
                preferred_tone=ToneStyle.FRIENDLY,
                preferred_detail=DetailLevel.COMPREHENSIVE,
                preferred_templates=["teacher", "standard"],
                patience=0.9,
                expertise_level=0.4
            ),
            "creative_writer": UserProfile(
                user_id="creative_writer",
                preferred_tone=ToneStyle.CASUAL,
                preferred_detail=DetailLevel.DETAILED,
                preferred_templates=["creative", "teacher"],
                patience=0.7,
                expertise_level=0.5
            )
        }
    
    def _create_query_bank(self) -> List[Query]:
        """Create sample queries for simulation."""
        return [
            Query("What is machine learning?", QueryType.FACTUAL, 0.3),
            Query("Explain quantum computing", QueryType.FACTUAL, 0.7, requires_reasoning=True),
            Query("Write a poem about AI", QueryType.CREATIVE, 0.5),
            Query("Create a story about robots", QueryType.CREATIVE, 0.6),
            Query("Analyze this sales data trend", QueryType.ANALYTICAL, 0.7, requires_reasoning=True),
            Query("What patterns do you see here?", QueryType.ANALYTICAL, 0.5),
            Query("Write a Python function for sorting", QueryType.CODING, 0.4, requires_examples=True),
            Query("Debug this code snippet", QueryType.CODING, 0.6),
            Query("How are you today?", QueryType.CONVERSATIONAL, 0.1),
            Query("What do you think about this?", QueryType.CONVERSATIONAL, 0.3),
        ]
    
    def get_random_query(self) -> Tuple[Query, str]:
        """Get a random query and user for interaction."""
        query = random.choice(self.query_bank)
        user_id = random.choice(list(self.user_profiles.keys()))
        return query, user_id
    
    def simulate_response(self, query: Query, template_id: str, 
                         tone: ToneStyle, detail: DetailLevel,
                         use_examples: bool, use_cot: bool) -> Dict[str, float]:
        """Simulate LLM response quality based on prompt configuration."""
        
        # Base quality from template-query match
        optimal_templates = self.optimal_mappings.get(query.query_type, ["standard"])
        if template_id in optimal_templates:
            base_quality = np.random.uniform(0.7, 0.95)
        else:
            base_quality = np.random.uniform(0.4, 0.7)
        
        # Bonus for using examples when needed
        if query.requires_examples and use_examples:
            base_quality += 0.1
        
        # Bonus for chain-of-thought on complex queries
        if query.requires_reasoning and use_cot:
            base_quality += 0.15
        elif not query.requires_reasoning and use_cot:
            base_quality -= 0.05  # Unnecessary CoT can hurt simple queries
        
        # Add noise
        quality = np.clip(base_quality + np.random.normal(0, 0.1), 0, 1)
        
        # Calculate cost
        template = PROMPT_TEMPLATES[template_id]
        cost = template.estimated_cost
        if use_examples:
            cost *= 1.2
        if use_cot:
            cost *= 1.3
        
        return {
            "quality": quality,
            "cost": cost,
            "latency": cost * np.random.uniform(0.8, 1.2)  # Simulated latency
        }
    
    def get_reward(self, query: Query, user_id: str, template_id: str,
                   tone: ToneStyle, detail: DetailLevel,
                   use_examples: bool, use_cot: bool,
                   cost_weight: float = 0.2) -> Tuple[float, Dict]:
        """
        Calculate reward combining user satisfaction and cost efficiency.
        
        Reward = satisfaction - cost_weight * normalized_cost
        """
        # Simulate response
        response_metrics = self.simulate_response(
            query, template_id, tone, detail, use_examples, use_cot
        )
        
        # Get user satisfaction
        user = self.user_profiles[user_id]
        satisfaction = user.get_satisfaction(
            template_id, tone, detail, response_metrics["quality"]
        )
        
        # Calculate reward
        normalized_cost = response_metrics["cost"] / 2.0  # Normalize to ~0-1 range
        reward = satisfaction - cost_weight * normalized_cost
        
        # Store interaction
        interaction = {
            "query_type": query.query_type.value,
            "user_id": user_id,
            "template": template_id,
            "tone": tone.value,
            "detail": detail.value,
            "use_examples": use_examples,
            "use_cot": use_cot,
            "quality": response_metrics["quality"],
            "satisfaction": satisfaction,
            "cost": response_metrics["cost"],
            "reward": reward
        }
        self.interaction_history.append(interaction)
        
        return reward, interaction


# Create the simulation environment
sim_env = SimulatedLLMEnvironment()

print("✅ Simulated environment created!")
print(f"   👥 {len(sim_env.user_profiles)} user profiles")
print(f"   📋 {len(sim_env.query_bank)} sample queries")
print(f"   📝 {len(PROMPT_TEMPLATES)} prompt templates")

## 4. Multi-Armed Bandit for Prompt Template Selection

The simplest RL approach: treat each prompt template as an "arm" and learn which templates work best for different query types.

In [ ]:
class MultiArmedBandit(ABC):
    """Abstract base class for bandit algorithms."""
    
    def __init__(self, n_arms: int, arm_names: List[str] = None):
        self.n_arms = n_arms
        self.arm_names = arm_names or [f"arm_{i}" for i in range(n_arms)]
        self.counts = np.zeros(n_arms)
        self.values = np.zeros(n_arms)
        self.history = []
    
    @abstractmethod
    def select_arm(self) -> int:
        """Select an arm to pull."""
        pass
    
    def update(self, arm: int, reward: float):
        """Update estimates after receiving reward."""
        self.counts[arm] += 1
        n = self.counts[arm]
        value = self.values[arm]
        # Incremental mean update
        self.values[arm] = value + (reward - value) / n
        self.history.append({"arm": arm, "reward": reward})
    
    def get_best_arm(self) -> int:
        """Return the arm with highest estimated value."""
        return np.argmax(self.values)


class EpsilonGreedyBandit(MultiArmedBandit):
    """Epsilon-Greedy Multi-Armed Bandit."""
    
    def __init__(self, n_arms: int, epsilon: float = 0.1, arm_names: List[str] = None):
        super().__init__(n_arms, arm_names)
        self.epsilon = epsilon
    
    def select_arm(self) -> int:
        if np.random.random() < self.epsilon:
            return np.random.randint(self.n_arms)
        return self.get_best_arm()


class UCBBandit(MultiArmedBandit):
    """Upper Confidence Bound (UCB1) Bandit."""
    
    def __init__(self, n_arms: int, c: float = 2.0, arm_names: List[str] = None):
        super().__init__(n_arms, arm_names)
        self.c = c
        self.total_counts = 0
    
    def select_arm(self) -> int:
        self.total_counts += 1
        
        # If any arm hasn't been tried, try it
        for arm in range(self.n_arms):
            if self.counts[arm] == 0:
                return arm
        
        # UCB formula
        ucb_values = self.values + self.c * np.sqrt(
            np.log(self.total_counts) / self.counts
        )
        return np.argmax(ucb_values)


class ThompsonSamplingBandit(MultiArmedBandit):
    """Thompson Sampling Bandit (Beta-Bernoulli)."""
    
    def __init__(self, n_arms: int, arm_names: List[str] = None):
        super().__init__(n_arms, arm_names)
        self.alpha = np.ones(n_arms)  # Successes + 1
        self.beta = np.ones(n_arms)   # Failures + 1
    
    def select_arm(self) -> int:
        samples = np.random.beta(self.alpha, self.beta)
        return np.argmax(samples)
    
    def update(self, arm: int, reward: float):
        super().update(arm, reward)
        # Update Beta distribution parameters
        if reward > 0.5:  # Treat as success
            self.alpha[arm] += 1
        else:
            self.beta[arm] += 1


class PromptTemplateBandit:
    """Uses bandit algorithm to select optimal prompt templates."""
    
    def __init__(self, templates: Dict[str, PromptTemplate], 
                 algorithm: str = "thompson"):
        self.templates = templates
        self.template_ids = list(templates.keys())
        self.n_templates = len(self.template_ids)
        
        # Create bandits for each query type
        self.bandits: Dict[QueryType, MultiArmedBandit] = {}
        for query_type in QueryType:
            if algorithm == "epsilon_greedy":
                self.bandits[query_type] = EpsilonGreedyBandit(
                    self.n_templates, epsilon=0.1, arm_names=self.template_ids
                )
            elif algorithm == "ucb":
                self.bandits[query_type] = UCBBandit(
                    self.n_templates, c=2.0, arm_names=self.template_ids
                )
            else:  # thompson
                self.bandits[query_type] = ThompsonSamplingBandit(
                    self.n_templates, arm_names=self.template_ids
                )
    
    def select_template(self, query_type: QueryType) -> str:
        """Select a template for the given query type."""
        bandit = self.bandits[query_type]
        arm = bandit.select_arm()
        return self.template_ids[arm]
    
    def update(self, query_type: QueryType, template_id: str, reward: float):
        """Update bandit after receiving feedback."""
        arm = self.template_ids.index(template_id)
        self.bandits[query_type].update(arm, reward)
    
    def get_best_templates(self) -> Dict[str, str]:
        """Get the best template for each query type."""
        result = {}
        for query_type in QueryType:
            bandit = self.bandits[query_type]
            best_arm = bandit.get_best_arm()
            result[query_type.value] = self.template_ids[best_arm]
        return result


print("✅ Bandit algorithms defined:")
print("   🎰 Epsilon-Greedy")
print("   🎰 Upper Confidence Bound (UCB1)")
print("   🎰 Thompson Sampling")

## 5. Q-Learning for Full Prompt Configuration

Q-Learning allows us to learn optimal combinations of template, tone, detail level, and other prompt parameters.

In [ ]:
@dataclass
class PromptAction:
    """Represents a complete prompt configuration action."""
    template_id: str
    tone: ToneStyle
    detail: DetailLevel
    use_examples: bool
    use_chain_of_thought: bool
    
    def to_tuple(self) -> tuple:
        return (self.template_id, self.tone.value, self.detail.value, 
                self.use_examples, self.use_chain_of_thought)
    
    @classmethod
    def from_tuple(cls, t: tuple) -> 'PromptAction':
        return cls(
            template_id=t[0],
            tone=ToneStyle(t[1]),
            detail=DetailLevel(t[2]),
            use_examples=t[3],
            use_chain_of_thought=t[4]
        )


class PromptQLearning:
    """Q-Learning agent for prompt optimization."""
    
    def __init__(self, templates: Dict[str, PromptTemplate],
                 learning_rate: float = 0.1,
                 discount_factor: float = 0.95,
                 epsilon: float = 0.2,
                 epsilon_decay: float = 0.995,
                 min_epsilon: float = 0.05):
        self.templates = templates
        self.template_ids = list(templates.keys())
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.min_epsilon = min_epsilon
        
        # Q-table: state -> action -> value
        # State: (query_type, user_id) or just query_type for simpler version
        self.q_table: Dict[str, Dict[tuple, float]] = defaultdict(lambda: defaultdict(float))
        
        # Generate all possible actions
        self.actions = self._generate_all_actions()
        
        # Training history
        self.reward_history = []
        self.epsilon_history = []
    
    def _generate_all_actions(self) -> List[PromptAction]:
        """Generate all possible prompt configurations."""
        actions = []
        for template_id in self.template_ids:
            for tone in ToneStyle:
                for detail in DetailLevel:
                    for use_examples in [True, False]:
                        for use_cot in [True, False]:
                            # Skip invalid combinations
                            template = self.templates[template_id]
                            if use_cot and not template.supports_chain_of_thought:
                                continue
                            if use_examples and not template.supports_examples:
                                continue
                            actions.append(PromptAction(
                                template_id, tone, detail, use_examples, use_cot
                            ))
        return actions
    
    def get_state(self, query: Query, user_id: str = None) -> str:
        """Convert query/user to state representation."""
        # Simple state: just query type
        # Can be extended to include user_id for personalization
        if user_id:
            return f"{query.query_type.value}_{user_id}"
        return query.query_type.value
    
    def select_action(self, state: str, training: bool = True) -> PromptAction:
        """Select action using epsilon-greedy policy."""
        if training and np.random.random() < self.epsilon:
            # Explore: random action
            return random.choice(self.actions)
        else:
            # Exploit: best known action
            state_q = self.q_table[state]
            if not state_q:
                return random.choice(self.actions)
            
            best_action_tuple = max(state_q.keys(), key=lambda a: state_q[a])
            return PromptAction.from_tuple(best_action_tuple)
    
    def update(self, state: str, action: PromptAction, reward: float, 
               next_state: str = None):
        """Update Q-value using Q-learning update rule."""
        action_tuple = action.to_tuple()
        old_value = self.q_table[state][action_tuple]
        
        # For prompt optimization, we typically don't have a "next state"
        # in the traditional sense, so we use a simplified update
        if next_state and self.q_table[next_state]:
            max_next = max(self.q_table[next_state].values())
        else:
            max_next = 0
        
        # Q-learning update
        new_value = old_value + self.learning_rate * (
            reward + self.discount_factor * max_next - old_value
        )
        self.q_table[state][action_tuple] = new_value
        
        # Decay epsilon
        self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)
        
        self.reward_history.append(reward)
        self.epsilon_history.append(self.epsilon)
    
    def get_best_action(self, state: str) -> Optional[PromptAction]:
        """Get the best action for a state without exploration."""
        state_q = self.q_table[state]
        if not state_q:
            return None
        best_action_tuple = max(state_q.keys(), key=lambda a: state_q[a])
        return PromptAction.from_tuple(best_action_tuple)
    
    def get_policy_summary(self) -> pd.DataFrame:
        """Summarize learned policy."""
        rows = []
        for state, actions in self.q_table.items():
            if actions:
                best_action_tuple = max(actions.keys(), key=lambda a: actions[a])
                best_q = actions[best_action_tuple]
                action = PromptAction.from_tuple(best_action_tuple)
                rows.append({
                    "state": state,
                    "template": action.template_id,
                    "tone": action.tone.value,
                    "detail": action.detail.value,
                    "examples": action.use_examples,
                    "cot": action.use_chain_of_thought,
                    "q_value": best_q
                })
        return pd.DataFrame(rows)


print("✅ Q-Learning agent defined")
print(f"   📊 Action space size: {len(PromptQLearning(PROMPT_TEMPLATES).actions)} possible configurations")

## 6. Training and Evaluation

Let's train both bandit and Q-learning agents and compare their performance.

In [ ]:
def train_bandit(env: SimulatedLLMEnvironment, n_iterations: int = 1000,
                 algorithm: str = "thompson") -> Tuple[PromptTemplateBandit, List[float]]:
    """Train a bandit agent on template selection."""
    bandit = PromptTemplateBandit(PROMPT_TEMPLATES, algorithm=algorithm)
    rewards = []
    
    for i in range(n_iterations):
        # Get random query and user
        query, user_id = env.get_random_query()
        
        # Select template using bandit
        template_id = bandit.select_template(query.query_type)
        
        # Use default settings for other parameters
        reward, _ = env.get_reward(
            query, user_id, template_id,
            tone=ToneStyle.FORMAL,
            detail=DetailLevel.MODERATE,
            use_examples=query.requires_examples,
            use_cot=query.requires_reasoning
        )
        
        # Update bandit
        bandit.update(query.query_type, template_id, reward)
        rewards.append(reward)
    
    return bandit, rewards


def train_qlearning(env: SimulatedLLMEnvironment, n_iterations: int = 2000,
                    personalized: bool = False) -> Tuple[PromptQLearning, List[float]]:
    """Train a Q-learning agent on full prompt configuration."""
    agent = PromptQLearning(PROMPT_TEMPLATES, epsilon=0.3, epsilon_decay=0.998)
    rewards = []
    
    for i in range(n_iterations):
        # Get random query and user
        query, user_id = env.get_random_query()
        
        # Get state
        state = agent.get_state(query, user_id if personalized else None)
        
        # Select action
        action = agent.select_action(state, training=True)
        
        # Get reward
        reward, _ = env.get_reward(
            query, user_id, action.template_id,
            tone=action.tone,
            detail=action.detail,
            use_examples=action.use_examples,
            use_cot=action.use_chain_of_thought
        )
        
        # Update Q-values
        agent.update(state, action, reward)
        rewards.append(reward)
    
    return agent, rewards


def evaluate_random_baseline(env: SimulatedLLMEnvironment, 
                            n_iterations: int = 500) -> List[float]:
    """Evaluate random template selection as baseline."""
    rewards = []
    template_ids = list(PROMPT_TEMPLATES.keys())
    
    for _ in range(n_iterations):
        query, user_id = env.get_random_query()
        template_id = random.choice(template_ids)
        tone = random.choice(list(ToneStyle))
        detail = random.choice(list(DetailLevel))
        
        reward, _ = env.get_reward(
            query, user_id, template_id,
            tone=tone, detail=detail,
            use_examples=random.choice([True, False]),
            use_cot=random.choice([True, False])
        )
        rewards.append(reward)
    
    return rewards


print("✅ Training functions defined")

In [ ]:
# Reset environment
sim_env = SimulatedLLMEnvironment()

print("🎯 Training Prompt Optimization Agents...")
print("="*60)

# Train bandit agents with different algorithms
print("\n📊 Training Multi-Armed Bandits...")
bandit_thompson, rewards_thompson = train_bandit(sim_env, n_iterations=1000, algorithm="thompson")
bandit_ucb, rewards_ucb = train_bandit(sim_env, n_iterations=1000, algorithm="ucb")
bandit_epsilon, rewards_epsilon = train_bandit(sim_env, n_iterations=1000, algorithm="epsilon_greedy")

# Train Q-learning agents
print("\n📊 Training Q-Learning Agents...")
qlearning_basic, rewards_qlearning = train_qlearning(sim_env, n_iterations=2000, personalized=False)
qlearning_personal, rewards_qlearning_personal = train_qlearning(sim_env, n_iterations=2000, personalized=True)

# Baseline
print("\n📊 Evaluating Random Baseline...")
rewards_random = evaluate_random_baseline(sim_env, n_iterations=500)

print("\n✅ Training Complete!")
print("="*60)

## 7. Results Visualization

In [ ]:
def moving_average(data, window=50):
    """Calculate moving average."""
    return pd.Series(data).rolling(window=window, min_periods=1).mean()

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle("Prompt Optimization with Reinforcement Learning", fontsize=16, fontweight='bold')

# Plot 1: Bandit Algorithm Comparison
ax1 = axes[0, 0]
ax1.plot(moving_average(rewards_thompson), label='Thompson Sampling', alpha=0.8)
ax1.plot(moving_average(rewards_ucb), label='UCB', alpha=0.8)
ax1.plot(moving_average(rewards_epsilon), label='Epsilon-Greedy', alpha=0.8)
ax1.axhline(y=np.mean(rewards_random), color='gray', linestyle='--', label='Random Baseline')
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Reward (Moving Avg)')
ax1.set_title('Multi-Armed Bandit: Template Selection')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Q-Learning Performance
ax2 = axes[0, 1]
ax2.plot(moving_average(rewards_qlearning, window=100), label='Q-Learning (Basic)', alpha=0.8)
ax2.plot(moving_average(rewards_qlearning_personal, window=100), label='Q-Learning (Personalized)', alpha=0.8)
ax2.axhline(y=np.mean(rewards_random), color='gray', linestyle='--', label='Random Baseline')
ax2.set_xlabel('Iteration')
ax2.set_ylabel('Reward (Moving Avg)')
ax2.set_title('Q-Learning: Full Prompt Configuration')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Final Performance Comparison
ax3 = axes[1, 0]
methods = ['Random', 'Thompson', 'UCB', 'ε-Greedy', 'Q-Learn', 'Q-Learn\n(Personal)']
final_rewards = [
    np.mean(rewards_random),
    np.mean(rewards_thompson[-200:]),
    np.mean(rewards_ucb[-200:]),
    np.mean(rewards_epsilon[-200:]),
    np.mean(rewards_qlearning[-400:]),
    np.mean(rewards_qlearning_personal[-400:])
]
colors = ['gray', '#2ecc71', '#3498db', '#e74c3c', '#9b59b6', '#f39c12']
bars = ax3.bar(methods, final_rewards, color=colors, edgecolor='black')
ax3.set_ylabel('Average Reward')
ax3.set_title('Final Performance Comparison')
ax3.axhline(y=np.mean(rewards_random), color='gray', linestyle='--', alpha=0.5)

# Add value labels on bars
for bar, val in zip(bars, final_rewards):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{val:.3f}', ha='center', va='bottom', fontsize=9)

# Plot 4: Q-Learning Exploration vs Exploitation
ax4 = axes[1, 1]
ax4.plot(qlearning_basic.epsilon_history, label='Epsilon (Exploration Rate)', color='#e74c3c')
ax4.set_xlabel('Iteration')
ax4.set_ylabel('Epsilon')
ax4.set_title('Q-Learning: Exploration Decay')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n📊 Performance Summary")
print("="*60)
print(f"{'Method':<25} {'Mean Reward':<15} {'Std Dev':<15} {'Improvement'}")
print("-"*60)
baseline = np.mean(rewards_random)
for name, rewards in [
    ('Random Baseline', rewards_random),
    ('Thompson Sampling', rewards_thompson[-200:]),
    ('UCB', rewards_ucb[-200:]),
    ('Epsilon-Greedy', rewards_epsilon[-200:]),
    ('Q-Learning (Basic)', rewards_qlearning[-400:]),
    ('Q-Learning (Personal)', rewards_qlearning_personal[-400:])
]:
    mean_r = np.mean(rewards)
    std_r = np.std(rewards)
    improvement = ((mean_r - baseline) / baseline) * 100
    print(f"{name:<25} {mean_r:<15.4f} {std_r:<15.4f} {improvement:+.1f}%")

## 8. Analyzing Learned Policies

Let's examine what the RL agents learned about optimal prompt configurations.

In [ ]:
# Analyze Bandit learned preferences
print("🎰 Multi-Armed Bandit: Learned Template Preferences")
print("="*60)
best_templates = bandit_thompson.get_best_templates()
print("\nBest template per query type (Thompson Sampling):")
for query_type, template in best_templates.items():
    template_name = PROMPT_TEMPLATES[template].name
    print(f"   {query_type:<15} → {template_name}")

# Compare with ground truth
print("\n📋 Ground Truth (Optimal Templates):")
for query_type, templates in sim_env.optimal_mappings.items():
    template_names = [PROMPT_TEMPLATES[t].name for t in templates]
    print(f"   {query_type.value:<15} → {', '.join(template_names)}")

# Analyze Q-Learning learned policy
print("\n\n🧠 Q-Learning: Learned Full Prompt Configurations")
print("="*60)
policy_df = qlearning_basic.get_policy_summary()
if not policy_df.empty:
    print(policy_df.to_string(index=False))
else:
    print("No policy learned yet.")

In [ ]:
# Visualize learned policy as heatmap
print("\n📊 Q-Learning Policy Visualization")
print("="*60)

# Create policy matrix for visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Template preferences by query type
ax1 = axes[0]
query_types = [qt.value for qt in QueryType]
template_ids = list(PROMPT_TEMPLATES.keys())

# Build preference matrix from Q-values
preference_matrix = np.zeros((len(query_types), len(template_ids)))
for i, qt in enumerate(QueryType):
    state = qt.value
    if state in qlearning_basic.q_table:
        for action_tuple, q_val in qlearning_basic.q_table[state].items():
            template_idx = template_ids.index(action_tuple[0])
            preference_matrix[i, template_idx] = max(preference_matrix[i, template_idx], q_val)

# Normalize for visualization
if preference_matrix.max() > 0:
    preference_matrix = preference_matrix / preference_matrix.max()

sns.heatmap(preference_matrix, xticklabels=template_ids, yticklabels=query_types,
            annot=True, fmt='.2f', cmap='YlGnBu', ax=ax1)
ax1.set_title('Learned Template Preferences by Query Type')
ax1.set_xlabel('Template')
ax1.set_ylabel('Query Type')

# Plot 2: Personalized preferences (by user)
ax2 = axes[1]
user_ids = list(sim_env.user_profiles.keys())

# Build user preference matrix
user_pref_matrix = np.zeros((len(user_ids), len(template_ids)))
for i, user_id in enumerate(user_ids):
    for qt in QueryType:
        state = f"{qt.value}_{user_id}"
        if state in qlearning_personal.q_table:
            for action_tuple, q_val in qlearning_personal.q_table[state].items():
                template_idx = template_ids.index(action_tuple[0])
                user_pref_matrix[i, template_idx] += max(0, q_val)

# Normalize
row_sums = user_pref_matrix.sum(axis=1, keepdims=True)
row_sums[row_sums == 0] = 1
user_pref_matrix = user_pref_matrix / row_sums

sns.heatmap(user_pref_matrix, xticklabels=template_ids, yticklabels=user_ids,
            annot=True, fmt='.2f', cmap='YlOrRd', ax=ax2)
ax2.set_title('Personalized Template Preferences by User')
ax2.set_xlabel('Template')
ax2.set_ylabel('User Profile')

plt.tight_layout()
plt.show()

## 9. Production Integration Example

Here's how you would integrate the trained RL agent into a production prompt optimization system.

In [ ]:
class ProductionPromptOptimizer:
    """Production-ready prompt optimizer with RL-based selection."""
    
    def __init__(self, templates: Dict[str, PromptTemplate],
                 pretrained_agent: PromptQLearning = None):
        self.templates = templates
        self.agent = pretrained_agent or PromptQLearning(templates)
        self.feedback_buffer = []
        self.online_learning = True
        
    def optimize_prompt(self, query_text: str, query_type: QueryType, 
                       user_id: str = None) -> Tuple[str, PromptAction]:
        """Generate optimized prompt for a query."""
        # Create query object
        query = Query(
            text=query_text,
            query_type=query_type,
            complexity=0.5,  # Could be estimated from query
            requires_examples=query_type == QueryType.CODING,
            requires_reasoning=query_type == QueryType.ANALYTICAL
        )
        
        # Get state and select action
        state = self.agent.get_state(query, user_id)
        action = self.agent.select_action(state, training=False)
        
        # Render the optimized prompt
        template = self.templates[action.template_id]
        optimized_prompt = template.render(
            query=query_text,
            tone=action.tone,
            detail=action.detail,
            chain_of_thought=action.use_chain_of_thought
        )
        
        return optimized_prompt, action
    
    def record_feedback(self, query_type: QueryType, action: PromptAction,
                       user_id: str, feedback_score: float):
        """Record user feedback for online learning."""
        self.feedback_buffer.append({
            'query_type': query_type,
            'action': action,
            'user_id': user_id,
            'feedback': feedback_score
        })
        
        # Online learning update
        if self.online_learning:
            state = f"{query_type.value}_{user_id}" if user_id else query_type.value
            self.agent.update(state, action, feedback_score)
    
    def get_optimization_stats(self) -> Dict[str, Any]:
        """Get statistics about prompt optimization."""
        if not self.feedback_buffer:
            return {"message": "No feedback recorded yet"}
        
        df = pd.DataFrame(self.feedback_buffer)
        return {
            "total_interactions": len(df),
            "avg_feedback": df['feedback'].mean(),
            "feedback_trend": df['feedback'].rolling(10).mean().tolist()[-10:],
            "most_used_template": df['action'].apply(lambda a: a.template_id).mode()[0]
        }


# Example usage
optimizer = ProductionPromptOptimizer(PROMPT_TEMPLATES, pretrained_agent=qlearning_personal)

# Demo: Optimize different queries
print("🚀 Production Prompt Optimizer Demo")
print("="*60)

demo_queries = [
    ("What is the capital of France?", QueryType.FACTUAL, "casual_user"),
    ("Write me a haiku about coding", QueryType.CREATIVE, "creative_writer"),
    ("Explain the complexity of quicksort", QueryType.ANALYTICAL, "tech_expert"),
    ("Help me debug this Python error", QueryType.CODING, "student"),
]

for query_text, query_type, user_id in demo_queries:
    prompt, action = optimizer.optimize_prompt(query_text, query_type, user_id)
    print(f"\n📝 Query: {query_text}")
    print(f"   User: {user_id}")
    print(f"   Selected: {action.template_id} | {action.tone.value} | {action.detail.value}")
    print(f"   Features: examples={action.use_examples}, cot={action.use_chain_of_thought}")
    print(f"\n   Generated Prompt Preview:")
    print(f"   {prompt[:200]}...")

## 10. Summary

This notebook demonstrated how **Reinforcement Learning** can be applied to **prompt optimization** in agentic workflows.

### Key Techniques Covered

| Technique | Use Case | Complexity |
|-----------|----------|------------|
| **Multi-Armed Bandit** | Template selection | Low - good for quick A/B testing |
| **Thompson Sampling** | Exploration-exploitation balance | Medium - probabilistic selection |
| **UCB** | Regret minimization | Medium - confidence-based exploration |
| **Q-Learning** | Full prompt configuration | High - learns optimal combinations |
| **Personalized Q-Learning** | User-specific optimization | High - adapts to individual preferences |

### What Can Be Optimized

1. **Prompt Template Selection** - Which persona/instruction works best
2. **Tone & Style** - Formal, casual, technical, friendly
3. **Detail Level** - Brief to comprehensive responses
4. **Few-shot Examples** - When to include examples
5. **Chain-of-Thought** - When reasoning steps help
6. **Cost vs Quality** - Balance API costs with response quality

### Reward Signals for Production

- **Explicit Feedback**: User ratings, thumbs up/down
- **Implicit Feedback**: Follow-up questions (negative), task completion (positive)
- **Task Success**: Did the response solve the user's problem?
- **Engagement**: Time spent, copy actions, shares
- **Cost Efficiency**: API tokens used, latency

### Next Steps

1. **Integrate with Real LLMs**: Replace simulation with actual API calls
2. **Add Contextual Features**: Include query embeddings for better state representation
3. **Implement RLHF**: Train a reward model from human preferences
4. **A/B Testing Framework**: Deploy with proper experiment tracking
5. **Online Learning**: Continuously improve from production feedback

In [ ]:
print("\n🎉 Prompt Optimization with Reinforcement Learning - Complete!")
print("\nThis notebook demonstrated:")
print("  • Multi-Armed Bandits for template selection (Thompson, UCB, ε-Greedy)")
print("  • Q-Learning for full prompt configuration optimization")
print("  • Personalized learning based on user preferences")
print("  • Simulated reward modeling with user feedback")
print("  • Production integration patterns")
print("\n💡 Key Insight: RL enables prompts to continuously improve")
print("   based on real user feedback, not just static engineering!")